<h1><center>ERM with DNN under penalty of Equalized Odds</center></h1>

We implement here a regular Empirical Risk Minimization (ERM) of a Deep Neural Network (DNN) penalized to enforce an Equalized Odds constraint. More formally, given a dataset of size $n$ consisting of context features $x$, target $y$ and a sensitive information $a$ to protect, we want to solve
$$
\text{argmin}_{h\in\mathcal{H}}\frac{1}{n}\sum_{i=1}^n \ell(y_i, h(x_i)) + \lambda \chi^2|_1
$$
where $\ell$ is for instance the MSE and the penalty is
$$
\chi^2|_1 = \left\lVert\chi^2\left(\hat{\pi}(h(x)|y, a|y), \hat{\pi}(h(x)|y)\otimes\hat{\pi}(a|y)\right)\right\rVert_1
$$
where $\hat{\pi}$ denotes the empirical density estimated through a Gaussian KDE.

### Imports

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import numpy as np

from examples.data_loading import read_dataset

### The dataset

We use here the _communities and crimes_ dataset that can be found on the UCI Machine Learning Repository (http://archive.ics.uci.edu/ml/datasets/communities+and+crime). Non-predictive information, such as city name, state... have been removed and the file is at the arff format for ease of loading.

In [2]:
sys.path.append(os.path.abspath(os.path.join('../..')))

In [3]:
x_train, y_train, a_train, x_test, y_test, a_test = read_dataset(name='crimes', fold=1)
n, d = x_train.shape

### The Deep Neural Network

We define a very simple DNN for regression here

In [4]:
class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 50
        self.first = nn.Linear(input_size, size)
        self.fc = nn.Linear(size, size)
        self.last = nn.Linear(size, num_classes)

    def forward(self, x):
        out = F.selu(self.first(x))
        out = F.selu(self.fc(out))
        out = self.last(out)
        out = torch.sigmoid(out) # NEW
        return out

### The fairness-penalized ERM

We now implement the full learning loop. The regression loss used is the quadratic loss with a L2 regularization and the fairness-inducing penalty.

In [5]:
def regularized_learning(x_train, y_train, a_train, model, fairness_metric, fairness_weight = 1.0, lr=1e-5, num_epochs=10, print_progress = True):    
    X = torch.tensor(x_train.astype(np.float32))
    A = torch.tensor(a_train.astype(np.float32))
    Y = torch.tensor(y_train.astype(np.float32))
    dataset = data_utils.TensorDataset(X, Y, A)
    dataset_loader = data_utils.DataLoader(dataset=dataset, batch_size=200, shuffle=True)

    # mse regression objective
    data_fitting_loss = nn.MSELoss()

    # stochastic optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)

    for j in range(num_epochs):
        if print_progress:
            print(f"EPOCH {j} started")
        for i, (x, y, a) in enumerate(dataset_loader):
            if print_progress:
                print(f"Batch {i} started")
            def closure():
                optimizer.zero_grad()
                prediction = model(x).flatten()
                loss = data_fitting_loss(prediction, y)
                loss += fairness_weight * fairness_metric(prediction, a, y)
                loss.backward()
                return loss

            optimizer.step(closure)
    return model

### Evaluation

For the evaluation on the test set, we compute two metrics: the MSE (accuracy) and HGR$|_\infty$ (fairness).

In [6]:
def evaluate(model, x, y, a, fairness_metric):
    X = torch.tensor(x.astype(np.float32))
    A = torch.Tensor(a.astype(np.float32))
    Y = torch.tensor(y.astype(np.float32))

    prediction = model(X).detach().flatten()
    loss = nn.MSELoss()(prediction, Y)
    discrimination = fairness_metric(prediction, A, Y)
    return loss.item(), discrimination

### Running everything together


In [7]:
def generate_fairness_metric(constrained_intervals_A, quantizition_intervals_Y, train, size_compensation = lambda x: np.sqrt(x)):
    
    def inside(num, endpoints):
        start, end = endpoints
        return start <= num and num < end
    
    def fairness_metric(Y_hat, A, Y):
        nd_losses = []
        n = len(Y_hat)
        for inter_Y in quantizition_intervals_Y:
            for inter_A in constrained_intervals_A:
                cnt_y_a = 0
                cnt_y = 0
                sum_y_yhat = 0
                sum_y_a_yhat = 0
                for i in range(len(Y_hat)): # could be sped up by combining with outer loop
                    if inside(Y[i], inter_Y):
                        cnt_y += 1
                        sum_y_yhat += Y_hat[i]
                        if inside(A[i], inter_A):
                            cnt_y_a += 1
                            sum_y_a_yhat += Y_hat[i]
                if cnt_y_a > 0 and cnt_y > 0:
                    curr_nd_loss = torch.abs(sum_y_a_yhat / cnt_y_a - sum_y_yhat / cnt_y) * size_compensation(cnt_y_a / n)
                    nd_losses.append(curr_nd_loss)
        nd_losses_torch = torch.tensor(nd_losses)
        return torch.mean(nd_losses_torch) if train else torch.max(nd_losses_torch)
    return fairness_metric

In [8]:
def generate_constrained_intervals(num_constrained_intervals):
    endpoints = np.linspace(0, 1, num_constrained_intervals + 1)
    constrained_intervals = []
    for i in range(len(endpoints) - 1):
        constrained_intervals.append((endpoints[i], endpoints[i + 1]))
    return constrained_intervals

In [ ]:
model = NetRegression(d, 1)
num_epochs = 20
lr = 1e-5
fairness_weight = 1
num_constrained_intervals = 10
intervals = generate_constrained_intervals(num_constrained_intervals)
fairness_metric_train = generate_fairness_metric(intervals, intervals, True)
fairness_metric_test = generate_fairness_metric(intervals, intervals, False)

model = regularized_learning(x_train, y_train, a_train, model=model, fairness_metric=fairness_metric_train, lr=lr, \
                             num_epochs=num_epochs, fairness_weight=fairness_weight)
mse, discrimination = evaluate(model, x_test, y_test, a_test, fairness_metric=fairness_metric_test)

EPOCH 0 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 1 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 2 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 3 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 4 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 5 started
Batch 0 started
Batch 1 started
Batch 2 started
Batch 3 started
Batch 4 started
Batch 5 started
Batch 6 started
Batch 7 started
Batch 8 started
EPOCH 6 started
Batch 0 started
Batch 1 

In [ ]:
print("MSE:{} Beta loss:{}".format(mse, discrimination))